In [ ]:
#@title Setup  { display-mode: "form", form-width: "30%" }

rclone = False #@param {type: "boolean" }
speedtest = False #@param {type: "boolean" }
ssh = True #@param {type: "boolean"}

# Install libraries
if rclone:
  !curl https://rclone.org/install.sh | sudo bash

if speedtest:
  !pip install speedtest-cli 2>&1
  !speedtest

if ssh:
  !wget https://github.com/tmate-io/tmate/releases/download/2.4.0/tmate-2.4.0-static-linux-i386.tar.xz &> /dev/null
  !mkdir -p tmate 
  !tar -xvf tmate-2.4.0-static-linux-i386.tar.xz --strip-components 1 &> /dev/null
  !rm tmate-2.4.0-static-linux-i386.tar.xz
  !./tmate -S /tmp/tmate.sock new-session -d &> /dev/null
  !sleep 1
  !./tmate -S /tmp/tmate.sock display -p 'Connect to this SSH address: #{tmate_ssh}' 